In [ ]:
!pip install -U sentence-transformers

In [89]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)

df = pd.read_csv("/content/drive/MyDrive/Full Projects/IMDb Semantic Similarity/imdb_semantic_search.csv")
df.head()

,Name,Story,Genres,Certificate
0,Bridgerton,"Wealth, lust, and betrayal set against the backdrop of Regency-era England, seen through the eyes of the powerful Bridgerton family.","Drama ,Romance",TV-MA
1,Cobra Kai,"Thirty years after their final confrontation at the 1984 All Valley Karate Tournament, Johnny Lawrence is at rock-bottom as an unemployed handyman haunted by his wasted life. However, when Johnny rescues a bullied kid, Miguel, from bullies, he is inspired to restart the notorious Cobra Kai dojo. However, this revitalization of his life and related misunderstandings find Johnny restarting his old rivalry with Daniel LaRousso, a successful businessman who may be happily married, but is missing...","Action ,Comedy ,Drama ,Sport",TV-14
2,The Mandalorian,"After the stories of Jango and Boba Fett, another warrior emerges in the Star Wars universe. The Mandalorian is set after the fall of the Empire and before the emergence of the First Order. We follow the travails of a lone gunfighter in the outer reaches of the galaxy far from the authority of the New Republic.","Action ,Adventure ,Sci-Fi",TV-14
3,Superstore,A look at the lives of employees at a big box store.,Comedy,TV-14
4,Game of Thrones,"In the mythical continent of Westeros, several powerful families fight for control of the Seven Kingdoms. As conflict erupts in the kingdoms of men, an ancient enemy rises once again to threaten them all. Meanwhile, the last heirs of a recently usurped dynasty plot to take back their homeland from across the Narrow Sea.","Action ,Adventure ,Drama ,Fantasy",TV-MA


In [90]:
tv_ma = df[df['Certificate'] == 'TV-MA']
all = df

In [91]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('bert-large-cased')
embedder.to('cuda')
embedder.max_seq_len = 512

tvma_stories = tv_ma['Story'].tolist()
tvma_titles = tv_ma['Name'].tolist()

titles = df['Name'].tolist()
ratings = df['Certificate'].tolist()
stories = df['Story'].tolist()

tvma_embeddings = embedder.encode(tvma_stories, convert_to_tensor=True)
story_embeddings = embedder.encode(stories, convert_to_tensor=True)

In [92]:
def semantic_search(story):
  title_list = []
  rating_list = []
  story_list = []
  score_list = []
  results = pd.DataFrame()



  # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
  top_k = min(10, len(story_embeddings))

  query_embedding = embedder.encode(query, convert_to_tensor=True)

  # We use cosine-similarity and torch.topk to find the highest 5 scores
  cos_scores = util.pytorch_cos_sim(query_embedding, story_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)

  print("\n\n======================")
  print("\tSTORY")
  print("======================\n")
  print(query)
  print("\n\n======================")
  print("    TOP 10 RESULTS")
  print("======================\n")

  for score, idx in zip(top_results[0], top_results[1]):
    title_list.append(titles[idx])
    rating_list.append(ratings[idx])
    story_list.append(stories[idx])
    score_list.append(score.cpu().numpy().flatten())

  results['Title'] = title_list
  results['Rating'] = rating_list
  results['Story'] = story_list
  results['Score'] = score_list

  return results

  
story = input("""Enter Story: """)
semantic_search(story)

Enter Story: The story of a former martial artist who after losing a prestigious tournament losses his way and becomes a wayward drifter, until that is, he stumbles across another young runaway called Santiago. Seeing a reflection of himself in the youth he decides to turn his life around for the better opening his own dojo. Together with Santiago they set out to win the tournament.


	STORY

The story of a former martial artist who after losing a prestigious tournament losses his way and becomes a wayward drifter, until that is, he stumbles across
another young runaway called Santiago. Seeing a reflection of himself in the youth he decides to turn his life around for the better opening his own dojo. Together
with Santiago they set out to win the tournament.


    TOP 10 RESULTS



,Title,Rating,Story,Score
0,Cobra Kai,TV-14,"Thirty years after their final confrontation at the 1984 All Valley Karate Tournament, Johnny Lawrence is at rock-bottom as an unemployed handyman haunted by his wasted life. However, when Johnny rescues a bullied kid, Miguel, from bullies, he is inspired to restart the notorious Cobra Kai dojo. However, this revitalization of his life and related misunderstandings find Johnny restarting his old rivalry with Daniel LaRousso, a successful businessman who may be happily married, but is missing...",[0.8475048]
1,My Name Is Earl,TV-PG,"Earl is a low-life who buys a winning lottery ticket, only to get hit by a car, losing the ticket in the process. He then realizes in the hospital that his bad luck is the result of karma in which fate punishes him for all the rotten things he's ever done in his life; therefore, he then decides to dedicate his life to making amends to all the people he has hurt in his life.",[0.825037]
2,Video Game High School,TV-14,"The story is about a down on his luck, young aspiring gamer. He has no friends. His cat hates him. His name is Brian. He loves playing video games. One day during a public match the number one gamer the ""Law"" arrives to do a little bit of showing off. Out of no where Brian manages to kill the ""Law"". It's like the shot heard around the world. Because of it and all the hype Brian gets invited to Video Game High School. It's about a kid and all the troubles of high school but in a very differen...",[0.8243076]
3,Arrow,TV-14,"Oliver Queen and his father are lost at sea when their luxury yacht sinks, apparently in a storm. His father dies, but Oliver survives for five years on an uncharted island and eventually returns home. But he wasn't alone on the island where he learned not only how to fight and survive but also of his father's corruption and unscrupulous business dealings. He returns to civilization a changed man, determined to put things right. He disguises himself with the hood of one of his mysterious isl...",[0.822947]
4,Community,TV-14,"A former lawyer attends a community college when it is discovered he faked his Bachelor's degree. In an attempt to get with a student in his Spanish class, he forms a Spanish study group. To his surprise, more people attend the study group and the group of misfits form an unlikely community.",[0.81606156]
5,Kevin (Probably) Saves the World,TV-14,"Once a successful and selfish Wall Streeter, Kevin Finn is on a downward spiral as he realizes he hates his life, which led to him trying to commit suicide. To regroup, Kevin heads back to his hometown of Taylor, Texas to live with his straight-laced, recently widowed, aeronautical engineer twin sister Amy Cabrera and her teenaged daughter Reese Cabrera, the latter who is not happy to see her uncle as he did not support them in their time of grief. Shortly after his arrival in Taylor, Kevin ...",[0.8142863]
6,Trollhunters,TV-Y7,"Jim Lake Jr. is an ordinary kid with a busy Mom and a yearning to have more adventure in his life. As it happens, Jim gets more than he wished when he finds a magic amulet under a bridge.calling to him. With that discovery, Jim finds himself recruited as the next. and first human, Troll Hunter, the champion of the Troll world. Together with his friend, Toby Domzalski, Jim discovers the world of Troll Market, where he begins his training under the tutelage of his new mentors, Blinky and AAARR...",[0.81057286]
7,Kickin' It,TV-Y7,"Set at the Bobby Wasabi Martial Arts Academy, a strip mall martial arts studio and worst dojo in the under-performing nationwide Bobby Wasabi chain. Revolves around a ragtag crew of lovable misfits who welcome a hotshot skateboarder named Jack who will teach them about life, karate, and how to just plain kick it.",[0.8079271]
8,Static Shock,TV-Y7,"In Dakota City, Virgil Hawkins is an ordinary kid who gets into big trouble, which gets him pressured into joining a street gang. That night, Virgil's gang has a maj